<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Example 3.2
import numpy as np
from scipy.interpolate import PchipInterpolator

# Define constants
TaC = 25          # Temperature (°C)
C = 0.5           # Step size
E = 0.5           # Maximum dI/dV error

# Define variables with initial conditions
Suns_initial = 0.045
Va = 31

# Placeholder for KYOCERA function (needs to be defined based on physical model)
def KYOCERA(Va, Suns, TaC):
    # Dummy implementation — replace with actual model
    return Suns * (0.02 * Va + 0.5)

Ia = KYOCERA(Va, Suns_initial, TaC)
Pa = Va * Ia
Vref_new = Va + C

Va_array = []
Pa_array = []
Pmax_array = []

Suns_matrix = np.array([
    [0, 0.1], [1, 0.2], [2, 0.3], [3, 0.3], [4, 0.5], [5, 0.6], [6, 0.7],
    [7, 0.8], [8, 0.9], [9, 1.0], [10, 1.1], [11, 1.2], [12, 1.3], [13, 1.4]
])

x = Suns_matrix[:, 0]  # Time data
y = Suns_matrix[:, 1]  # Solar radiation data

xi = np.arange(1, 201)  # Interpolation points
interp = PchipInterpolator(x, y)
yi = interp(xi)         # Interpolated values

for sample in range(14):
    # Read radiation data
    Suns = yi[sample]

    # Take a new measurement
    Va_new = Vref_new
    Ia_new = KYOCERA(Va, Suns, TaC)

    # Calculate the incremental in voltage and current
    deltaVa = Va_new - Va
    deltaIa = Ia_new - Ia

    if deltaVa == 0:
        if deltaIa == 0:
            Vref_new = Va_new  # No change
        elif deltaIa > 0:
            Vref_new = Va_new + C
        else:
            Vref_new = Va_new - C
    else:
        if abs(deltaIa / deltaVa + Ia_new / Va_new) <= E:
            Vref_new = Va_new  # No change
        else:
            if deltaIa / deltaVa > -Ia_new / Va_new + E:
                Vref_new = Va_new + C
            else:
                Vref_new = Va_new - C

    # Update variables
    Va = Va_new
    Ia = Ia_new
    Pa = Va * Ia

    Va_array.append(Va)
    Pa_array.append(Pa)
